reference: http://trap.ncirl.ie/1845/1/briangibbons.pdf

# 1. Import Packages

In [1]:
from urllib.request import urlopen
import json
#from bs4 import BeautifulSoup
#from lxml import html
#import requests
import pandas as pd
#import numpy as np
from openpyxl import load_workbook
#import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

# 2. Fuction

In [2]:
def writing_in_excel(path, df, sheet_name):
    writer = pd.ExcelWriter(path, datetime_format='m/d/yyyy')
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [3]:
def writing_sheets_in_excel(path, df, sheet_name):
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl', date_format='m/d/yyyy')
    writer.book = book
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [4]:
def drop_xy(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    
    #to_replace = [x for x in df if x.endswith('_x')]
    df.columns = [c.lower().replace('_x', '') for c in df.columns] 

In [5]:
def creating_excel_by_keys(path, html):
    path = path
    raw_html = urlopen(html)
    data = json.load(raw_html)
    keys = list(data.keys())
    i = 0
    while i < len(keys):
        try:
            name = keys[i]
            df1 = pd.DataFrame(data[keys[i]])
            writing_in_excel(path + name + '.xlsx', df1, name)
        except:
            pass
        i += 1


# Test if crawler works

In [39]:
# info about bootstrap (overall)
bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
bootstrap = json.load(bootstrap_html)

In [40]:
# info about one player

player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/423/")
player_raw = json.load(player_html)

In [41]:
bootstrap.keys()
player_raw.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

dict_keys(['fixtures', 'history', 'history_past'])

In [47]:
len(bootstrap['elements'])

589

In [15]:
# From Bootstrap

events = pd.DataFrame(bootstrap['events'])
#game_settings = pd.DataFrame(bootstrap['game_settings'])
phases =  pd.DataFrame(bootstrap['phases']) 
teams =  pd.DataFrame(bootstrap['teams']) 
#total_players = pd.DataFrame(bootstrap['total_players'])
elements =  pd.DataFrame(bootstrap['elements']) 
element_stats =  pd.DataFrame(bootstrap['element_stats']) 
element_types =  pd.DataFrame(bootstrap['element_types'])

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past = pd.DataFrame(player_raw['history_past'])

In [50]:
#events
#phases
#teams
#elements

#element_stats
#element_types



0      a
1      d
2      a
3      a
4      a
      ..
584    a
585    a
586    a
587    a
588    a
Name: status, Length: 589, dtype: object

In [26]:
##### From Player_raw ######

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past =  pd.DataFrame(player_raw['history_past']) 
    
#fixtures
#history.iloc
#history_past

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,2008/09,40386,40,40,20,471,1,0,0,6,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,2009/10,40386,40,38,12,241,0,1,0,3,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,2010/11,40386,40,38,1,121,0,0,0,3,...,0,1,0,0,0,0,0.0,0.0,0.0,0.0


# Defining a scraper

In [27]:
def bootstrap_scraper():
    
    # info about bootstrap (overall)
    bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
    bootstrap = json.load(bootstrap_html)
    
    events = pd.DataFrame(bootstrap['events'])
    #game_settings = pd.DataFrame(bootstrap['game_settings'])
    phases =  pd.DataFrame(bootstrap['phases']) 
    teams =  pd.DataFrame(bootstrap['teams']) 
    #total_players = pd.DataFrame(bootstrap['total_players'])
    elements =  pd.DataFrame(bootstrap['elements']) 
    element_stats =  pd.DataFrame(bootstrap['element_stats']) 
    element_types =  pd.DataFrame(bootstrap['element_types'])
    
    select = 'D'

    events.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019//events2019.csv')
    #game_settings.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\game_settings2019.xlsx')
    phases.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\phases2019.csv')
    teams.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\teams2019.csv')
    elements.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\elements2019.csv')
    element_stats.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_stats2019.csv')
    element_types.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_types2019.csv')


In [51]:
def player_data_scraper():
    
    i=1
    frames = []
    
    while i < 100:
        try:
            player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/" + str(i) + '/')            
            player_raw = json.load(player_html)
            
            history = player_raw['history']
            #bootstrap = bootstrap_raw['elements']
            history = pd.DataFrame(history)
            #bootstrap_df = pd.DataFrame(bootstrap)
            
            #player_df['position'] = bootstrap_df.loc[bootstrap_df['id'] == i, 'element_type']
            
            frames.append(history)
            result_df = pd.concat(frames)
            result_df = pd.DataFrame(result_df)
            #print(i)
            
        except:
            pass
            
    
        print (i)
        i += 1
    
    return result_df
    

# Run the scrapers

In [52]:
#bootstrap_scraper()
result_df = player_data_scraper()


select = 'D'
result_df.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\each_player_each_game.csv')

    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [37]:
result_df

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,10,13,0,False,2019-08-11T13:00:00Z,0,1,1,0,...,0,0.0,0.0,0.0,0.0,55,0,33117,0,0
1,1,11,5,0,True,2019-08-17T11:30:00Z,2,1,2,0,...,0,0.0,0.0,0.0,0.0,55,-5280,36709,2868,8148
2,1,24,10,0,False,2019-08-24T16:30:00Z,3,1,3,0,...,0,0.0,0.0,0.0,0.0,54,-6882,30975,534,7416
3,1,31,17,0,True,2019-09-01T15:30:00Z,2,2,4,0,...,0,0.0,0.0,0.0,0.0,54,-3872,28096,346,4218
4,1,49,18,0,False,2019-09-15T15:30:00Z,2,2,5,0,...,0,0.0,0.0,0.0,0.0,53,-2073,26902,581,2654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,9,161,11,0,True,2019-12-15T16:30:00Z,0,3,17,0,...,0,0.0,0.0,0.0,0.0,45,2,1620,35,33
17,9,174,8,0,False,2019-12-21T12:30:00Z,0,0,18,0,...,0,0.0,0.0,0.0,0.0,45,-10,1618,8,18
18,9,182,3,0,False,2019-12-26T15:00:00Z,1,1,19,0,...,0,0.0,0.0,0.0,0.0,45,-2,1617,9,11
19,9,191,6,0,True,2019-12-29T14:00:00Z,1,2,20,0,...,0,0.0,0.0,0.0,0.0,45,-1,1615,10,11


# Debugging
